In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import set_config; set_config(display='diagram')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics.pairwise import sigmoid_kernel, cosine_similarity, linear_kernel

from zytholic_project.base_model import BaseModel
from zytholic_project.evaluate import evaluate_proximity, get_recommendations, test_prediction

In [4]:
dfbrew = pd.read_csv("../raw_data/Beers_Breweries_and_Beer Reviews/breweries.csv")
dfbeer = pd.read_csv("../raw_data/beers_style_renamed.csv")
dftop = pd.read_csv("../raw_data/top_beer_info_style_renamed.csv")

In [21]:
dftop.head()

,name,key,style,style key,brewery,description,abv,ave rating,min ibu,max ibu,...,body,alcohol,bitter,sweet,sour,salty,fruits,hoppy,spices,malty
0,Amber,251,Altbier,8,Alaskan Brewing Co.,"Notes:Richly malty and long on the palate, wit...",5.3,3.65,25,50,...,32,9,47,74,33,0,33,57,8,111
1,Double Bag,252,Altbier,8,Long Trail Brewing Co.,"Notes:This malty, full-bodied double alt is al...",7.2,3.90,25,50,...,57,18,33,55,16,0,24,35,12,84
2,Long Trail Ale,253,Altbier,8,Long Trail Brewing Co.,Notes:Long Trail Ale is a full-bodied amber al...,5.0,3.58,25,50,...,37,6,42,43,11,0,10,54,4,62
3,Doppelsticke,254,Altbier,8,Uerige Obergärige Hausbrauerei,Notes:,8.5,4.15,25,50,...,55,31,47,101,18,1,49,40,16,119
4,Scurry,255,Altbier,8,Off Color Brewing,Notes:Just cause it's dark and German doesn't ...,5.3,3.67,25,50,...,69,10,63,120,14,0,19,36,15,218


In [ ]:
#read correspondance brewery
corres_xls = pd.read_csv('../assets/matching_brewery_names.csv')
corres_xls.set_index('bbr', inplace=True)
corres= corres_xls.to_dict()
#corres[0]

In [ ]:
dfbrew = dfbrew.rename(columns={"name": "brewery"})
dfbrew = dfbrew.rename(columns={"id": "brewery_id"})


dfbrewb =  pd.merge(dfbeer,dfbrew[['brewery_id','brewery']],how='left',on=['brewery_id'])

dftopbrew = pd.merge(dftop,dfbrewb[['name', 'brewery', 'state', 'country', 'retired']],
                     how='inner',on=['name','brewery'])

In [ ]:
working_df = dftopbrew.drop(['description', 'key', 'style key'], axis= 1).drop_duplicates()
print(working_df.shape)
working_df = working_df[working_df.retired == 'f']
working_df.shape
#working_df['style'] = [st.split(' - ')[0] for st in working_df['style']]

working_df_train, working_df_test = train_test_split(working_df, test_size=0.2, stratify=working_df['style'])


In [ ]:
working_df.sample(5)

# Pipeline

In [ ]:
tastes_features = working_df.select_dtypes(np.number).columns[2:]
tastes_features

pipe_style_country = make_pipeline(OneHotEncoder(sparse=False, handle_unknown='ignore'))
pipe_abv_rating = make_pipeline(MinMaxScaler())
pipe_taste_features = make_pipeline(MinMaxScaler())
pipe_state = make_pipeline(
    SimpleImputer(strategy='constant', fill_value=''),
    OneHotEncoder(sparse=False, handle_unknown='ignore')
)

preprocess = make_column_transformer(
    (pipe_style_country, ['style', 
                          #'country'
                         ]),
    #(pipe_state, ['state']),
    (pipe_abv_rating, ['abv', 'ave rating']),
    (pipe_taste_features, tastes_features)
)

In [ ]:
preprocess

In [ ]:
preprocess.fit(X_train)
X_train_proc = preprocess.transform(X_train)
X_test_proc = preprocess.transform(X_test)

In [ ]:
X_train_proc.shape, X_test_proc.shape

# Inertia calculations

In [ ]:

inertias2 = []
for k in range(30, 100):
    res = KMeans(n_clusters=k)
    res.fit(X)
    inertias.append(res.inertia_)

In [ ]:
plt.plot(range(3, 100),inertias)

In [ ]:
inertias2 = []
for k in range(10, 30):
    res = KMeans(n_clusters=k)
    res.fit(X)
    inertias2.append(res.inertia_)

In [ ]:
plt.plot(range(10, 30),inertias2)

In [ ]:
cluster = KMeans(n_clusters=20)
cluster.fit(X)

In [ ]:
X_test['group'] = cluster.predict(y)

In [ ]:
X_test.group.value_counts()

# BaseModel Class usage

In [ ]:
model = BaseModel()

model.get_data()

model.set_preprocess_pipeline()

model.process_data()


# compute sigmoid distance
sigmoid = sigmoid_kernel(
    model.X_train_proc,
    model.X_train_proc
)

#base_results = evaluate_proximity(model.X_train, sigmoid)

In [ ]:
inertias2 = []
for k in range(155, 256, 5):
    res = model.fit(clusts= k)
    inertias2.append(res.kmeans_fit.inertia_)

In [ ]:
plt.plot(range(155, 256, 5), inertias2)

In [ ]:
base_results.plot.scatter(x='style', y='matching_percent')

In [ ]:
base_results.describe().T

In [ ]:
df.sample(10)

In [ ]:
final_style.sort_values('matching_percent', ascending=False).tail(20)

In [ ]:
final_style.sort_values('matching_percent', ascending=False).head(20)

In [ ]:
def reformat_styles(working_df, ohe=True):
    """
    Simplify the columns 'style' of an input DF
    Converts various features insides style name to OHE features
    """
    # specific columns to implement
    features_to_implement = ['milk', 'old', 'dark', 'wild', 'pale', 'red', 'imperial']

    # Get matching table for styles names and format it
    style_xls = pd.read_excel('../assets/style_convert.xlsx')
    style_xls.columns = style_xls.iloc[0, :]
    style_xls = style_xls.iloc[1:, 1:]


    # creation of a dictionary to replace automatically
    style_dict = style_xls.set_index('Converted').to_dict()
    style_dict = style_dict['Simplified']
    style_dict

    #styles_test = working_df[['style']].drop_duplicates()

    working_df['simple_style'] = working_df['style'].replace(style_dict)
    
    # One-Hot-Encoding of featrues_to_implement
    if ohe:
        for feat in features_to_implement:
            working_df[feat] = [1 if feat in elm.lower() else 0 for elm in working_df['style']]
        
    working_df.rename(columns={'style':'original_style', 'simple_style':'style'}, inplace=True)

    return working_df

# Evaluation of average recommended style

In [ ]:
reformated_df = reformat_styles(working_df)

preprocess.fit(reformated_df)

X_train = preprocess.transform(reformated_df)

sigmoid = sigmoid_kernel(X_train, X_train)

results = evaluate_proximity(working_df, sigmoid)

Results with fearture_to_impelement OHE + Simple style

In [ ]:
results.plot.scatter(x='style', y='matching_percent')

In [ ]:
results.describe().T

Results without fearture_to_impelement OHE + Simple style

In [ ]:
reformated_df = reformat_styles(working_df, ohe=False)

preprocess.fit(reformated_df)

X_train = preprocess.transform(reformated_df)

sigmoid = sigmoid_kernel(X_train, X_train)

results = evaluate_proximity(working_df, sigmoid)

In [ ]:
results.plot.scatter(x='style', y='matching_percent')

In [ ]:
results.describe().T

In [ ]:
results['style'].nunique()

Notes:
    
    10 beers in recommendations might be to hight
    try to reduce id to 5
    
    Also increase number of samples to test from

In [ ]:
reformated_df = working_df_train
preprocess.fit(reformated_df)
X_train = preprocess.transform(reformated_df)
sigmoid = sigmoid_kernel(X_train, X_train)

base_results = evaluate_proximity(working_df_train,  n_recomm=10, tests=30, sim_matrix=sigmoid)

In [ ]:
base_results.plot.scatter(x='style', y='matching_percent')

In [ ]:
base_results.describe().T

In [ ]:
base_results

In [ ]:
reformated_df = reformat_styles(working_df_train)
preprocess.fit(reformated_df)
X_train = preprocess.transform(reformated_df)
sigmoid = sigmoid_kernel(X_train, X_train)

results = evaluate_proximity(working_df_train,  n_recomm=5, tests=30, sim_matrix=sigmoid)

In [ ]:
results.plot.scatter(x='style', y='matching_percent')

In [ ]:
results.describe().T

In [ ]:
reformated_df = reformat_styles(working_df_train, ohe=False)
preprocess.fit(reformated_df)
X_train = preprocess.transform(reformated_df)
sigmoid = sigmoid_kernel(X_train, X_train)

results = evaluate_proximity(working_df_train,  n_recomm=5, tests=30, sim_matrix=sigmoid)

In [ ]:
results.plot.scatter(x='style', y='matching_percent')

In [ ]:
results.describe().T

In [ ]:
X_train.shape

In [ ]:
working_df_train

# Comparison of similarity metrics

In [ ]:
model = BaseModel()

model.get_data()

model.set_preprocess_pipeline()

model.process_data()

In [ ]:
# compute sigmoid distance
sigmoid = sigmoid_kernel(
    model.X_train_proc,
    model.X_train_proc
)

sig_results = evaluate_proximity(model.X_train,  n_recomm=5, tests=30, sim_matrix=sigmoid)
sig_results.describe().T

In [ ]:
cosine = cosine_similarity(
    model.X_train_proc,
    model.X_train_proc
)

cosine_results = evaluate_proximity(model.X_train,  n_recomm=5, tests=30, sim_matrix=cosine)
cosine_results.describe().T

In [ ]:
linear = linear_kernel(
    model.X_train_proc,
    model.X_train_proc
)

linear_results = evaluate_proximity(model.X_train,  n_recomm=5, tests=30, sim_matrix=linear)
linear_results.describe().T

# Predictions for beers in the test set

In [ ]:
def test_prediction(model, n_recomm = 10):
    
    # Use the features to calculate kernel (calculate all distances at once?)
    sigmoid = cosine_similarity(model.X_test_proc, model.X_train_proc)

    matching_results = []
    substyles = []
    # Predict the closest beers in reference dataset
    for idx in range(sigmoid.shape[0]):
        distances = sigmoid[idx, :]
        sorted_distances = sorted(
                list(enumerate(distances)),
                key=lambda x:x[1],reverse=True)
        # get the top n predictions
        closest_items = sorted_distances[0:n_recomm+1]
        beers_indices = [i[0] for i in closest_items]
        
        # check original_style == predict_style percentage
        original_style = model.X_test["style"].iloc[idx]
        propositions = model.X_train.iloc[beers_indices, :]
        matching_percent = propositions[propositions['style'] == original_style].shape[0]\
            / propositions.shape[0] * 100
        matching_results.append(matching_percent)
        
        #compare substyle matching
        original_substyle = model.X_test.iloc[idx, -7:]
        comp = propositions.iloc[:, -7:]
        substyle_match = ((original_substyle == comp).sum()/ comp.shape[0]).min()
        substyles.append(substyle_match)
    
    results = model.X_test.copy()
    results['matching_percent'] = matching_results
    results['substyle'] = substyles
    return results

In [ ]:
results = test_prediction(model)


results.groupby('style')[['matching_percent', 'substyle']].agg('describe')

In [ ]:
a = model.X_train.iloc[0, -7:]

b = model.X_train.iloc[0:20, -7:]

In [ ]:
((a == b).sum() / b.shape[0])

In [ ]:
results = test_prediction(model)


results.groupby('style')[['matching_percent', 'matching_substyle']].agg('describe')


# Api Call Functions